In [16]:
import numpy as np
from numpy.linalg import inv

In [17]:
ZERO = 1e-8
np.set_printoptions(precision=4)

In [18]:
def simplex(A, b, c, base, logging=False):
    for iteracao in range(100):
        entrar, sair = None, None
        nao_base = [i for i in range(A.shape[1]) if i not in base]
        B = A[:, base]
        B_inv = inv(B)
        c_b = c[:, base]
        x_b = B_inv @ b
        z = (c_b @ x_b).item()
        ranges = calcular_ranges(B_inv, x_b)
        folgas = b - A[:, base] @ x_b

        p = c_b @ B_inv # Solução do dual
        s = c[:, nao_base] - p @ A[:, nao_base] # Custo reduzido 
        s_positivos_idx = np.where(s > ZERO)[1]  
        if s_positivos_idx.size == 0: # Solução ótima
            if logging:
                log_iteracao(iteracao, base, nao_base, ranges, s, entrar, sair, x_b, z, p)
            return z, base, x_b, p, folgas
        s_positivos = s[:, s_positivos_idx]

        maior_s = np.max(s_positivos)
        empatados_idx = np.where(np.abs(s_positivos - maior_s) <= ZERO)[1] # indices em s_positivos
        entrar = nao_base[s_positivos_idx[empatados_idx.min()]] # Regra de Bland
            
      # Teste da razão
        y = B_inv @ A[:, entrar].reshape(-1, 1)
        if not np.any(y > ZERO):
            print("problema ilimitado")
            return z, base, x_b, p, folgas
        razoes = np.divide(x_b[y > ZERO], y[y > ZERO]) 
        menor_razao = np.min(razoes) 
        empatados_idx = np.where(np.abs(razoes - menor_razao) <= ZERO)[0] # indices em razoes, converter de volta para indices da base
        sair_idx = np.where(y > ZERO)[0][empatados_idx.min()]  # Regra de Bland
        sair = base[sair_idx]
            
        if logging:
            log_iteracao(iteracao, base, nao_base, ranges, s, entrar, sair, x_b, z, p)
        
        base[sair_idx] = entrar

In [19]:
def calcular_ranges(B_inv, x_b):
    # sj * delta + x_b >= 0
    # delta >= - x_b / sj Quando sj > 0
    # delta <= - x_b / sj Quando sj < 0  (-|sj| delta >= -x_b * (-1)) 
    all_ranges = []
    x_b = x_b.flatten()
    
    for j in range(B_inv.shape[1]):
        Sj = B_inv[:, j].flatten()
        mask_valid = np.abs(Sj) > ZERO
        Sj_valid = Sj[mask_valid]
        x_b_valid = x_b[mask_valid]
        
        range_j = -x_b_valid / Sj_valid
        Sj_pos = Sj_valid > ZERO
        Sj_neg = Sj_valid < -ZERO  
        limite_superior = np.min(range_j[Sj_neg]).item() if np.any(Sj_neg) else np.inf # delta <= ranges_j[si < 0]
        limite_inferior = np.max(range_j[Sj_pos]).item() if np.any(Sj_pos) else -np.inf  # delta >= range_j[si > 0] 

        all_ranges.append((limite_inferior, limite_superior))

    return all_ranges

In [20]:
def simplex_duas_fases(A, b, c, tipo_restricoes, logging=False):
    qtde_restricoes = A.shape[0]
    qtde_variaveis = c.shape[1] # reais + folgas
    indices_artificiais = [i for i, op in enumerate(tipo_restricoes) if op in ('>=', '=')]
    qtde_artificiais = len(indices_artificiais)
    
    # Fase 1
    A_artificiais = np.zeros((qtde_restricoes, qtde_artificiais)) 
    for col, i in enumerate(indices_artificiais):
        A_artificiais[i, col] = 1
    A = np.hstack((A, A_artificiais))
    c_artificiais = np.hstack((np.zeros((1, qtde_variaveis)), -1*np.ones((1, qtde_restricoes)))) 
    base = list(range(A.shape[1] - qtde_restricoes, A.shape[1]))    

    z, base, x_b, p, folgas = simplex(A, b, c_artificiais, base)
    if abs(z) > ZERO:
        print(f"Problema inviável, valor da função objetivo na primeira fase: {z:.4f}")
        return
    if logging:  
        print(f"\n ### Duas fases - Base Encontrada: {base}")

    # Fase 2
    A = A[:, :qtde_variaveis]
    base = base[:qtde_variaveis]
    z, base, x_b, p, folgas = simplex(A, b, c, base, logging=logging)
    return z, base, x_b, p, folgas

In [21]:
def show_dual(A, b, c):
    c_dual = b.T
    b_dual = c.T
    A_dual = A.T
    
    print("\nDual")
    termos_obj = [f"{c_dual[0, i]}p{i+1}" for i in range(c_dual.shape[1])]
    print("min" + " " + " + ".join(termos_obj))
    print("st")
    for i, restricao in enumerate(A_dual):
        termos = [f"{coef}p{j+1}" for j, coef in enumerate(restricao)]
        print(f"{' + '.join(termos)} >= {b_dual[i].item()}")

    # return A_dual, b_dual, c_dual

### Funções Utilitárias

In [22]:
def log_iteracao(iteracao, base, nao_base, ranges, s, entrar, sair, x_b, z, p):
    print("\n\n-----------------------------------------------------")
    print(f"Iteração {iteracao+1}")
    print("Base:", [f"x{i+1}" for i in base])
    print("Não base:", [f"x{i+1}" for i in nao_base])
    print(f"Valor da função objetivo: {z:.4f}")
    ranges_str = [(f"{li:.4f}", f"{ls:.4f}") for li, ls in ranges]
    print(f"Ranges da mão direita (b): {ranges_str}")    
    print(f"Solução do dual: {p}")
    print(f"Custo relativo / pricing - s: {s}")
    print(f"Próximo a entrar na base: {"x" + str(entrar+1) if entrar is not None else "Nenhum"}")
    print(f"Próximo a sair da base: {"x" + str(sair+1) if sair is not None else "Nenhum"}")
    input()

def imprimir_resultado(z, base, x_b, p, c, folgas):
    print("\n################ Solução ótima encontrada ################")
    print(f"Valor da função objetivo: {z:.4f}")

    sol_primal = []
    for idx_base, i in enumerate(base):
        if i < c.shape[1] and c[0, i] != 0:  # variável real
            val = x_b[idx_base]
            sol_primal.append(f"x{i+1} = {val.item():.4f}")
            
    print(f"Solução do Primal: {sol_primal}")
    p = p[0]
    sol_dual = [f"y{i+1} = {p[i]:.4f}" for i in range(len(p))]
    print(f"Solução do Dual: {sol_dual}")

    escassos = np.where(np.abs(folgas) <= ZERO)[0]
    abundantes = np.where(folgas > ZERO)[0]
    print("\nRecursos escassos (sem folga):")
    for i in escassos:
        print(f"Restrição {i+1}: escasso (folga {folgas[i].item():.4f})")
    print("\nRecursos abundantes (com folga):")
    for i in abundantes:
        print(f"Restrição {i+1}: abundante (folga {folgas[i].item():.4f})")


In [23]:
def read_file(filepath):
    def process_coef(tokens):
        coef_list = []
        sinal = 1
        for token in tokens:
            if token == "+":
                sinal = 1
            elif token == "-":
                sinal = -1
            else:
                # Extrair coeficiente da variável
                i = 0
                while i < len(token) and (token[i].isdigit() or token[i] in [".", "-", "+"]):
                    i += 1
                coef_part = token[:i]
                if coef_part in ("+", ""):  # Caso seja "x1" ou "+x1"
                    coef = 1
                elif coef_part == "-":  # Caso seja -x1
                    coef = -1
                else:
                    coef = float(coef_part.strip())
                    
                coef_list.append(sinal * coef)
        return coef_list
    
    with open(filepath) as f:
        linhas = [linha.strip() for linha in f if linha.strip()]

    max_min = linhas[0].split(" ")[0].lower()
    mult_objetivo = 1 if max_min == "max" else -1
    linha_objetivo = linhas[0].replace("max", "").replace("min", "").strip()
    tokens_obj = linha_objetivo.strip().split()
    c = process_coef(tokens_obj)
    
    restricoes = []
    for linha in linhas[2:]:
        if "<=" in linha:
            tokens_restricao, b = linha.split("<=")
            op = "<="
        elif ">=" in linha:
            tokens_restricao, b = linha.split(">=")
            op = ">="
        elif "=" in linha:
            tokens_restricao, b = linha.split("=")
            op = "="
        tokens_restricao = tokens_restricao.strip().split()
        coef_restricao = process_coef(tokens_restricao)
        b_val = float(b.strip())
        restricoes.append((op, coef_restricao, b_val))

    restricoes.sort(key=lambda r: {">=": 0, "=": 1, "<=": 2}[r[0]])

    A = []
    b = []
    qtde_folgas = 0
    for idx, (op, coef, b_val) in enumerate(restricoes):
        linha = coef.copy()
        for i in range(qtde_folgas):
            linha.append(0)
        if op == "<=":
            linha.append(1)
            c.append(0)
            qtde_folgas += 1
        elif op == ">=":
            linha.append(-1)
            c.append(0)
            qtde_folgas += 1
        A.append(linha)
        b.append(b_val)

    qtde_variaveis = max(len(restricao) for restricao in A)
    for restricao in A:
        while len(restricao) < qtde_variaveis:
            restricao.append(0)

    tipo_restricoes = [op for op, _, _ in restricoes]  
    
    return np.array(A), np.array(b).reshape(-1, 1), mult_objetivo * np.array([c]), mult_objetivo, tipo_restricoes

### Uso


In [24]:
A, b, c, mult_objetivo, tipo_restricoes = read_file("input.txt")
print(f"A:\n{A}")
print(f"b:\n{b}")
print(f"c:\n{c}")
show_dual(A, b, c)

z, base, x_b, p, folgas = simplex_duas_fases(A, b, c, tipo_restricoes, logging=True)
z = z * mult_objetivo # Forma padrão MAX
imprimir_resultado(z, base, x_b, p, c, folgas)


A:
[[ 1.  2. -1.  0.]
 [ 1. -1.  0.  0.]
 [ 1.  1.  0.  1.]]
b:
[[ 8.]
 [ 2.]
 [10.]]
c:
[[2. 3. 0. 0.]]

Dual
min 8.0p1 + 2.0p2 + 10.0p3
st
1.0p1 + 1.0p2 + 1.0p3 >= 2.0
2.0p1 + -1.0p2 + 1.0p3 >= 3.0
-1.0p1 + 0.0p2 + 0.0p3 >= 0.0
0.0p1 + 0.0p2 + 1.0p3 >= 0.0

 ### Duas fases - Base Encontrada: [3, 1, 0]


-----------------------------------------------------
Iteração 1
Base: ['x4', 'x2', 'x1']
Não base: ['x3']
Valor da função objetivo: 14.0000
Ranges da mão direita (b): [('-6.0000', '6.0000'), ('-6.0000', '6.0000'), ('-4.0000', 'inf')]
Solução do dual: [[1.6667 0.3333 0.    ]]
Custo relativo / pricing - s: [[1.6667]]
Próximo a entrar na base: x3
Próximo a sair da base: x4




-----------------------------------------------------
Iteração 2
Base: ['x3', 'x2', 'x1']
Não base: ['x4']
Valor da função objetivo: 24.0000
Ranges da mão direita (b): [('-inf', '6.0000'), ('-12.0000', '8.0000'), ('-4.0000', 'inf')]
Solução do dual: [[ 0.  -0.5  2.5]]
Custo relativo / pricing - s: [[-2.5]]
Próximo a entrar na base: Nenhum
Próximo a sair da base: Nenhum



################ Solução ótima encontrada ################
Valor da função objetivo: 24.0000
Solução do Primal: ['x2 = 4.0000', 'x1 = 6.0000']
Solução do Dual: ['y1 = 0.0000', 'y2 = -0.5000', 'y3 = 2.5000']

Recursos escassos (sem folga):
Restrição 1: escasso (folga 0.0000)
Restrição 2: escasso (folga 0.0000)
Restrição 3: escasso (folga 0.0000)

Recursos abundantes (com folga):


:D
